Use notebook for congestion data and data-slice generation and plotting; no adhoc-analyses here
Rename to transmission_analysis; have defs that can be called, then convert this into def calls and test notebook.

Code improvements
* rerun with updated westerintnet
* add all new cols to wecc.branches or wecc.CongStatBase (e.g., distances between buses, zscore, pvalue etc) for use in plotting
* convert code to defs as useful
* run as test book, importing as needed
* be consistent with index types
* fix names to have xxx_yyy format

Analysis improvements: additional output cols in congestion data
* normalize counts to number of year total hours and compare to wecc.biz output
* normalize counts to number of direct neighbors
* normalize counts to total path that connects to the line

In [1]:
import pandas as pd
import numpy as np
from math import *
import os

import sys
sys.path.append("..")
#sys.path.append("./test")

import analyze
import transmission
import westernintnet
WI = westernintnet.WesternIntNet()

Loading sub
Loading bus2sub
Loading bus
Loading genbus
Loading branches
Loading resources
Loading net_generation
Load solar data
Load wind data
Load hydro data
Load demand data
Done loading
Loading sub
Loading bus2sub
Loading bus
Loading genbus
Loading branches
Loading resources
Loading net_generation
Load solar data
Load wind data
Load hydro data
Load demand data
Done loading


In [2]:
data_dir = 'U:\\src\\PostREISE\\postreise\\analyze\\TransmissionLineAnalysis\\data\\'

#Reading takes about 15 minutes
data_dir = 'U:\\src\\PostREISE\\postreise\\analyze\\TransmissionLineAnalysis\\data\\'
sftp = analyze.setup_server_connection("c:/users/mlamherr/.ssh/id_rsa")  
scenario_list = analyze.get_scenario_file_from_server(sftp)  
scenario = 'parallel_western_scenario04'  

if not os.path.exists(data_dir + '/' + 'PF_' + scenario + '.pkl'):
    PF = analyze.get_PF(sftp,scenario_list,scenario)  
    PF.to_pickle(data_dir + '/' + 'PF_' + scenario + '.pkl')  


In [3]:
PF = pd.read_pickle(data_dir + 'PF_parallel_western_scenario04.pkl')

In [5]:
WI.PF_base = PF
WI.branches.index = PF.columns

#Create Capacity column; Note that rateA == 0 represents low impedance lines
#Denoting rateA==0 is simply a signal to MatPower that there are no current constraints on the line.
WI.branches.loc[WI.branches.rateA != 0, 'Capacity'] = WI.branches['rateA']
WI.branches.loc[WI.branches.rateA == 0, 'Capacity'] = 99999.

#Normalize power flow values to line capacity
WI.congestion_base = WI.PF_base.div(WI.branches['Capacity']).apply(np.abs)  

#Create congestion statistics file
WI.congestion_stats_base = transmission.generate_cong_stats(WI.congestion_base,'congestion_stats_base')

In [6]:
WI.congestion_stats_base.sort_values('zscore', ascending=False).head()

,Capacity,hutil1,hutil0p9-1,hutil0p8-0p9,hutil0p75-0p8,hutil>=0p9,hutil>=0p8,hutil>=0p75,dist,zscore,pvalue
line,,,,,,,,,,,
11484,269,0,0,0,8784,0,0,8784,0.0,1936.714640,0.0
2140,173,0,0,0,8781,0,0,8781,0.0,1936.051645,0.0
7205,14,0,0,0,8767,0,0,8767,0.0,1932.957667,0.0
12556,738,0,0,0,8212,0,0,8212,0.0,1810.303541,0.0
12558,738,0,0,0,7753,0,0,7753,0.0,1708.865265,0.0


In [7]:
WI.congestion_stats_base.loc[WI.congestion_stats_base['zscore']<=1.96].sort_values('zscore', ascending=False).head()

,Capacity,hutil1,hutil0p9-1,hutil0p8-0p9,hutil0p75-0p8,hutil>=0p9,hutil>=0p8,hutil>=0p75,dist,zscore,pvalue
line,,,,,,,,,,,
2952,11,0,0,0,28,0,0,28,0.0,1.652440,0.049222
10319,22,0,0,0,27,0,0,27,0.0,1.431442,0.076152
10505,183,0,0,0,25,0,0,25,0.0,0.989445,0.161223
9408,24,0,0,0,25,0,0,25,0.0,0.989445,0.161223
9286,268,0,0,0,25,0,0,25,0.0,0.989445,0.161223


In [8]:
WI.congestion_stats_base.columns


Index(['Capacity', 'hutil1', 'hutil0p9-1', 'hutil0p8-0p9', 'hutil0p75-0p8',
       'hutil>=0p9', 'hutil>=0p8', 'hutil>=0p75', 'dist', 'zscore', 'pvalue'],
      dtype='object')

In [1]:
PF.head()

NameError: name 'PF' is not defined